#1. Installations

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu bleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5782 sha256=8bf8dc960feb291cfe57f90004f095db764214345304ffd342ee0f23e4bf3af0
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.7 MB/s eta 0:00:00


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.37.2


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="pytorch")

# 2. Preprocessing the data


In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "tr-en")
metric1 = load_metric("sacrebleu")
metric2 = load_metric("bleu")

Generating train split:   0%|          | 0/205756 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

<ipython-input-8-c6e10a4d9cdb>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
metric2

Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_length': reference_length
Examples

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-tatoeba-en-tr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/794k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/838k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "tr_TR"

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "tr"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/205756 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets.column_names)
tokenized_datasets["test"][0]

{'train': ['translation', 'input_ids', 'attention_mask', 'labels'], 'validation': ['translation', 'input_ids', 'attention_mask', 'labels'], 'test': ['translation', 'input_ids', 'attention_mask', 'labels']}


{'translation': {'en': 'In 911 Call, Professor Admits to Shooting Girlfriend',
  'tr': "Profesör 911'i Arayarak Kız Arkadaşını Öldürdüğünü İtiraf Etti"},
 'input_ids': [323, 14674, 2319, 3, 5766, 30086, 9, 11, 47395, 47004, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [5629,
  21966,
  85,
  2699,
  9594,
  1730,
  8196,
  164,
  6032,
  13921,
  11100,
  36873,
  0]}

# 3. Baseline model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_base = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/150M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

predictions = []
references = []

input_ids_list = tokenized_datasets["test"]["input_ids"]
input_ids_tensor_list = [torch.tensor(input_ids) for input_ids in input_ids_list]

padded_input_ids_tensor = pad_sequence(input_ids_tensor_list, batch_first=True, padding_value=tokenizer.pad_token_id)

batch_size = 10
num_samples = len(padded_input_ids_tensor)
num_batches = (num_samples + batch_size - 1) // batch_size

for i in range(num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, num_samples)

    batch_input_ids = padded_input_ids_tensor[start_index:end_index]

    output = model_base.generate(batch_input_ids)

    with open('/content/drive/MyDrive/predictions_baseline.txt', 'a') as f:
      for item in output.tolist():
        f.write("%s\n" % item)

    for o in output:
        predictions.append(tokenizer.decode(o, skip_special_tokens=True))

    print(i)


In [ ]:
file_path = '/content/drive/MyDrive/predictions_baseline.txt'

preds = []

with open(file_path, 'r') as f:
    for line in f:
        preds.append(line.strip())

labels = tokenized_datasets["test"]["labels"]

preds1 = [pred.replace('[', '').replace(']', '').replace(',', '') for pred in preds]
preds = [[int(num) for num in pred.split()] for pred in preds1]

print(preds[0])

[59993, 21966, 85, 11072, 5629, 3, 556, 10152, 26974, 3882, 801, 2, 0, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993]


In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

if isinstance(preds, tuple):
    preds = preds[0]
decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

# Replace -100 in the labels as we can't decode them.
#labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Some simple post-processing
decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)



In [ ]:
print(decoded_preds[:2])
#print(decoded_labels[1])

["911'i arayan Profesör, kız arkadaşını vurduğunu itiraf etti.", '911 çağrısında sesi biraz titrekti, üniversite profesörü Shannon Lamb polise kız arkadaşını ve memurların evlerine gitmesi gerektiğini söyledi.']


In [ ]:
results = metric1.compute(predictions=decoded_preds, references=decoded_labels)
print(list(results.keys()))
print(round(results["score"], 1))

['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
21.4


In [ ]:
predictions_split = [pred.split() for pred in decoded_preds]
labels_split = [[ref.split() for ref in refs] for refs in decoded_labels]

print(predictions_split[:2])
print(labels_split[:2])

[["911'i", 'arayan', 'Profesör,', 'kız', 'arkadaşını', 'vurduğunu', 'itiraf', 'etti.'], ['911', 'çağrısında', 'sesi', 'biraz', 'titrekti,', 'üniversite', 'profesörü', 'Shannon', 'Lamb', 'polise', 'kız', 'arkadaşını', 've', 'memurların', 'evlerine', 'gitmesi', 'gerektiğini', 'söyledi.']]
[[['Profesör', "911'i", 'Arayarak', 'Kız', 'Arkadaşını', 'Öldürdüğünü', 'İtiraf', 'Etti']], [["911'i", 'arayan', 'Üniversite', 'Profesörü', 'Shannon', 'Lamb,', 'polise', 'kız', 'arkadaşını', 'silahla', 'vurduğunu', 've', 'polisin', 'evlerin', 'gelmesi', 'gerektiğini', 'sesi', 'yalnızca', 'biraz', 'titrek', 'şekilde', 'söyledi.']]]


In [ ]:
results = metric2.compute(predictions=predictions_split, references=labels_split)
print(list(results.keys()))
print(results["bleu"])

['bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length']
0.17750402068730622


# 3. Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/150M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    label_smoothing_factor=0.1,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.434800,3.023981


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[59993]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[59993]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=12860, training_loss=2.471535628394428, metrics={'train_runtime': 908.8343, 'train_samples_per_second': 226.396, 'train_steps_per_second': 14.15, 'total_flos': 3180452648583168.0, 'train_loss': 2.471535628394428, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[59993]], 'forced_eos_token_id': 0}


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1708841157.957c108288e6.640.0:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/300M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/verach3n/opus-tatoeba-en-tr-finetuned-en-to-tr/commit/1bd7932aeddd9aa20d418d8514aed8be98abaf95', commit_message='End of training', commit_description='', oid='1bd7932aeddd9aa20d418d8514aed8be98abaf95', pr_url=None, pr_revision=None, pr_num=None)

# 4. Evaluating the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("verach3n/opus-tatoeba-en-tr-finetuned-en-to-tr")

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

predictions = []
references = []

input_ids_list = tokenized_datasets["test"]["input_ids"]
input_ids_tensor_list = [torch.tensor(input_ids) for input_ids in input_ids_list]

padded_input_ids_tensor = pad_sequence(input_ids_tensor_list, batch_first=True, padding_value=tokenizer.pad_token_id)

batch_size = 10
num_samples = len(padded_input_ids_tensor)
num_batches = (num_samples + batch_size - 1) // batch_size

for i in range(num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, num_samples)

    batch_input_ids = padded_input_ids_tensor[start_index:end_index]

    output = model.generate(batch_input_ids)

    with open('/content/drive/MyDrive/predictions_finetuned_2.txt', 'a') as f:
      for item in output.tolist():
        f.write("%s\n" % item)

    for o in output:
        predictions.append(tokenizer.decode(o, skip_special_tokens=True))

    print(i)


In [ ]:
file_path = '/content/drive/MyDrive/predictions_finetuned_2.txt'

preds = []

with open(file_path, 'r') as f:
    for line in f:
        preds.append(line.strip())

labels = tokenized_datasets["test"]["labels"]

preds1 = [pred.replace('[', '').replace(']', '').replace(',', '') for pred in preds]
preds = [[int(num) for num in pred.split()] for pred in preds1]

print(preds[0])

[59993, 1959, 4344, 26702, 2982, 3, 5629, 1730, 27278, 5894, 10739, 11100, 36873, 0, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993, 59993]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(labels)

[[5629, 21966, 85, 2699, 9594, 1730, 8196, 164, 6032, 13921, 11100, 36873, 0], [21966, 85, 11072, 10708, 5629, 991, 21909, 21148, 3, 8843, 556, 10152, 18410, 26974, 25, 12911, 1530, 446, 8462, 1698, 3283, 2534, 313, 17252, 126, 499, 576, 2, 0], [21148, 3, 45, 19128, 389, 264, 8783, 185, 139, 2462, 25, 2987, 5737, 163, 25, 7834, 3475, 11788, 2013, 1432, 3475, 6442, 18057, 125, 3445, 5722, 11297, 2, 0], [9203, 1761, 3364, 6118, 30, 573, 3920, 4, 109, 42626, 24575, 25, 2498, 14, 37, 11367, 255, 1709, 20222, 455, 10632, 14, 37, 3900, 96, 45, 6408, 1172, 3, 9190, 56, 365, 327, 9504, 45997, 3, 3364, 4, 306, 79, 33377, 25, 93, 61, 16571, 106, 202, 1889, 375, 27421, 139, 11474, 391, 13716, 21148, 4, 109, 14448, 1628, 11297, 7595, 90, 2, 0], [23604, 3, 14802, 4, 833, 14772, 12148, 7510, 36, 345, 45311, 3270, 2691, 21148, 4, 109, 3, 12911, 37939, 7128, 27592, 9095, 13678, 479, 88, 15, 4, 109, 39, 21148, 831, 9502, 185, 1997, 3100, 1290, 1168, 28531, 18798, 3643, 29240, 14646, 358, 3588, 4986, 58

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

if isinstance(preds, tuple):
    preds = preds[0]
decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

# Replace -100 in the labels as we can't decode them.
#labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Some simple post-processing
decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)



In [ ]:
print(decoded_preds[:2])
#print(decoded_labels[1])

['1111 Çağrısında, Profesör Kız Arkadaşı Vurduğunu İtiraf Etti', "11 Eylül'de yaptığı bir telefon konuşmasında, üniversite öğretim üyesiShannon Lamb polise kız arkadaşını ve polis memurlarını evine gitmek için vurması gerektiğini söyledi."]


In [ ]:
results = metric1.compute(predictions=decoded_preds, references=decoded_labels)
print(list(results.keys()))
print(round(results["score"], 1))

['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
20.3


In [ ]:
predictions_split = [pred.split() for pred in decoded_preds]
labels_split = [[ref.split() for ref in refs] for refs in decoded_labels]

print(predictions_split[:2])
print(labels_split[:2])

[['1111', 'Çağrısında,', 'Profesör', 'Kız', 'Arkadaşı', 'Vurduğunu', 'İtiraf', 'Etti'], ['11', "Eylül'de", 'yaptığı', 'bir', 'telefon', 'konuşmasında,', 'üniversite', 'öğretim', 'üyesiShannon', 'Lamb', 'polise', 'kız', 'arkadaşını', 've', 'polis', 'memurlarını', 'evine', 'gitmek', 'için', 'vurması', 'gerektiğini', 'söyledi.']]
[[['Profesör', "911'i", 'Arayarak', 'Kız', 'Arkadaşını', 'Öldürdüğünü', 'İtiraf', 'Etti']], [["911'i", 'arayan', 'Üniversite', 'Profesörü', 'Shannon', 'Lamb,', 'polise', 'kız', 'arkadaşını', 'silahla', 'vurduğunu', 've', 'polisin', 'evlerin', 'gelmesi', 'gerektiğini', 'sesi', 'yalnızca', 'biraz', 'titrek', 'şekilde', 'söyledi.']]]


In [ ]:
results = metric2.compute(predictions=predictions_split, references=labels_split)
print(list(results.keys()))
print(results["bleu"])

['bleu', 'precisions', 'brevity_penalty', 'length_ratio', 'translation_length', 'reference_length']
0.16597852204901112
